In [2]:
%%capture

!pip install tensorflow==2.3.0
!pip install sagemaker-experiments

#### Imports 

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sagemaker.tensorflow.serving import TensorFlowModel
from sagemaker.multidatamodel import MultiDataModel
from tensorflow.keras.datasets import cifar10
from sagemaker.tensorflow import TensorFlow
from sagemaker.inputs import TrainingInput
from sagemaker import get_execution_role
from tensorflow.keras import utils
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from datetime import datetime
import tensorflow as tf
import numpy as np
import sagemaker
import logging
import boto3
import time
import os

#### Setup Logger

In [4]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())
!python --version

Python 3.7.10


In [5]:
logger.info(f'[Using TensorFlow version: {tf.__version__}]')
logger.info(f'[Using SageMaker version: {sagemaker.__version__}]')

[Using TensorFlow version: 2.3.0]
[Using SageMaker version: 2.94.0]


#### Seed for Reproducability

In [6]:
SEED = 123
np.random.seed(SEED)
tf.random.set_seed(SEED)

#### Create Roles, Sessions and Data Locations

In [21]:
role = get_execution_role()
session = boto3.Session()
sagemaker_session = sagemaker.Session()

s3 = session.resource('s3')
TF_FRAMEWORK_VERSION = '2.3.0'
BUCKET = sagemaker.Session().default_bucket()
PREFIX = 'cv-models'
MONITORING_FOLDER = 'DEMO-tf2-ModelMonitor'

### Train - CIFAR-10 Image Classification

<p align="justify">First, we will train a Convolutional Neural Network (CNN) model to classify images from the CIFAR-10 dataset. Image classification is the task of assigning a label to an image, from a predefined set of categories. CIFAR-10 is an established CV dataset used for object recognition. It is a subset of the 80 Million Tiny Images dataset and consists of 60,000 (32x32) color images containing 1 of 10 object classes, with 6,000 images per class.</p>

#### a) Load Data

The first step is to load the pre-shuffled CIFAR-10 dataset into our train and test objects. Luckily, Keras provides the CIFAR dataset for us to load using the `load_data()` method. All we have to do is import keras.datasets and then load the data.

In [13]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [14]:
logger.info(f'X_train Shape: {X_train.shape}')
logger.info(f'y_train Shape: {y_train.shape}')
logger.info(f'X_test Shape : {X_test.shape}')
logger.info(f'y_test Shape : {y_test.shape}')

X_train Shape: (50000, 32, 32, 3)
y_train Shape: (50000, 1)
X_test Shape : (10000, 32, 32, 3)
y_test Shape : (10000, 1)


#### c) Data Preparation

##### Rescale 
Rescales the images by dividing the pixel values by 255: [0,255] ⇒ [0,1]

In [15]:
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

##### One Hot Encode Target Labels
One-hot encoding is a process by which categorical variables are converted into a numeric form. One-hot encoding converts the (1 × n) label vector to a label matrix of dimensions (10 × n), where n is the number of sample images. So, if we have 1,000 images in our dataset, the label vector will have the dimensions (1 × 1000). After one-hot encoding, the label matrix dimensions will be (1000 × 10). That’s why, when we define our network architecture in the next step, we will make the output softmax layer contain 10 nodes, where each node represents the probability of each class we have.

In [16]:
num_classes = len(np.unique(y_train))
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

##### Split Data
Break original train set further into train and validation sets.

In [17]:
X_train, X_validation = X_train[500:], X_train[:500]
y_train, y_validation = y_train[500:], y_train[:500]

##### Save to Local

Create a local `data/cifar_10` directory to save the datasets.

In [190]:
DATASET_PATH = './data/cifar_10'

In [191]:
os.makedirs(DATASET_PATH, exist_ok=True)

Save train, validation and test sets to local `data` directory

In [192]:
np.save(f'{DATASET_PATH}/X_train.npy', X_train)
np.save(f'{DATASET_PATH}/y_train.npy', y_train)
np.save(f'{DATASET_PATH}/X_validation.npy', X_validation)
np.save(f'{DATASET_PATH}/y_validation.npy', y_validation)
np.save(f'{DATASET_PATH}/X_test.npy', X_test)
np.save(f'{DATASET_PATH}/y_test.npy', y_test)

##### Copy Datasets to S3
Copy train, validation and test sets from the local dir to S3, since SageMaker expects datasets to be in S3 for training.

In [15]:
!aws s3 cp ./{DATASET_PATH}/X_train.npy s3://{BUCKET}/{PREFIX}/cifar_10/train/
!aws s3 cp ./{DATASET_PATH}/y_train.npy s3://{BUCKET}/{PREFIX}/cifar_10/train/
!aws s3 cp ./{DATASET_PATH}/X_validation.npy s3://{BUCKET}/{PREFIX}/cifar_10/validation/
!aws s3 cp ./{DATASET_PATH}/y_validation.npy s3://{BUCKET}/{PREFIX}/cifar_10/validation/
!aws s3 cp ./{DATASET_PATH}/X_test.npy s3://{BUCKET}/{PREFIX}/cifar_10/test/
!aws s3 cp ./{DATASET_PATH}/y_test.npy s3://{BUCKET}/{PREFIX}/cifar_10/test/

upload: data/cifar_10/X_train.npy to s3://sagemaker-us-east-1-949263681218/cv-models/cifar_10/train/X_train.npy
upload: data/cifar_10/y_train.npy to s3://sagemaker-us-east-1-949263681218/cv-models/cifar_10/train/y_train.npy
upload: data/cifar_10/X_validation.npy to s3://sagemaker-us-east-1-949263681218/cv-models/cifar_10/validation/X_validation.npy
upload: data/cifar_10/y_validation.npy to s3://sagemaker-us-east-1-949263681218/cv-models/cifar_10/validation/y_validation.npy
upload: data/cifar_10/X_test.npy to s3://sagemaker-us-east-1-949263681218/cv-models/cifar_10/test/X_test.npy
upload: data/cifar_10/y_test.npy to s3://sagemaker-us-east-1-949263681218/cv-models/cifar_10/test/y_test.npy


# Create Training Inputs

In [40]:
train_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar_10/train', 
                            distribution='FullyReplicated', 
                            content_type='npy')
validation_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar_10/validation', 
                                 distribution='FullyReplicated', 
                                 content_type='npy')
test_input = TrainingInput(s3_data=f's3://{BUCKET}/{PREFIX}/cifar_10/test', 
                           distribution='FullyReplicated', 
                           content_type='npy')

In [41]:
inputs = {'train': train_input, 'val': validation_input, 'test': test_input}

#### e) Define Model Architecture & create Training Script

We will build a small CNN consisting of three convolutional layers and two dense layers.<br>
<b>Note:</b> We will use the ReLU activation function for all the hidden layers. In the last dense layer, we will use a softmax activation function with 10 nodes to return an array of 10 probability scores (summing to 1). Each score will be the probability that the current image belongs to our 10 image classes.

# Prepare a Experiment Tracker

In [42]:
sm = boto3.client('sagemaker')

In [ ]:
from smexperiments.experiment import Experiment
cifar_experiment = Experiment.create(
    experiment_name="cifar-10-dataset-experiment", 
    description="objects", 
    sagemaker_boto_client=sm)

In [ ]:
from smexperiments.trial import Trial
for num_hidden_channel in [32]:
    trial_name = f"cnn-training-job-{num_hidden_channel}-hidden-channels-{int(time.time())}"
    cnn_trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=cifar_experiment.experiment_name,
        sagemaker_boto_client=sm,
    )
    cnn_trial.add_trial_component(tracker.trial_component)

#### f) Create a TensorFlow Estimator & fit the Model

In [43]:
from sagemaker.tuner import ContinuousParameter,  IntegerParameter, HyperparameterTuner
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(1e-4, 1, scaling_type="Logarithmic")
#     "epochs": IntegerParameter(1,3),
}

In [44]:
model_name = 'cifar-10'
hyperparameters = {'epochs': 1}
estimator_parameters = {'entry_point':'cifar_train.py',
                        'instance_type': 'ml.m5.2xlarge',
                        'instance_count': 1,
                        'model_dir': '/opt/ml/model',
                        'role': role,
#                         'hyperparameters': hyperparameters,
                        'output_path': f's3://{BUCKET}/{PREFIX}/cifar_10/out',
                        'base_job_name': f'mme-cv-{model_name}',
                        'framework_version': TF_FRAMEWORK_VERSION,
                        'py_version': 'py37',
                        'script_mode': True}
model = TensorFlow(**estimator_parameters)

In [45]:
objective_metric_name = "loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "loss", "Regex": "loss : ([0-9\\.]+)"}]

tuner = HyperparameterTuner(
    model,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=9,
    max_parallel_jobs=5,
    objective_type=objective_type,
)

In [51]:
tuner.fit(inputs, wait=True)

.....................................................................................................................................................................................!


In [50]:
cnn_training_job_name = "cnn-training-job-{}".format(int(time.time()))
print(cnn_training_job_name)
model.fit(inputs, job_name=cnn_training_job_name,
        experiment_config={
            "ExperimentName": "cifar-10-dataset-experiment", 
            "TrialName": 'cnn-training-job-32-hidden-channels-1655871250',
            "TrialComponentDisplayName": "Training",
        })

cnn-training-job-1656309139
2022-06-27 05:52:20 Starting - Starting the training job...
2022-06-27 05:52:43 Starting - Preparing the instances for trainingProfilerReport-1656309139: InProgress
......
2022-06-27 05:53:48 Downloading - Downloading input data...
2022-06-27 05:54:14 Training - Downloading the training image......
2022-06-27 05:55:21 Uploading - Uploading generated training model
2022-06-27 05:55:21 Failed - Training job failed
2022-06-27 05:55:05,747 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2022-06-27 05:55:05,754 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-27 05:55:06,164 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-27 05:55:06,264 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-27 05:55:06,280 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-06-2

UnexpectedStatusException: Error for Training job cnn-training-job-1656309139: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/local/bin/python3.7 cifar_train.py --model_dir /opt/ml/model"
Traceback (most recent call last):
  File "cifar_train.py", line 30, in <module>
    logger.addHandler(logging.StreamHandler(sys.stdout))
NameError: name 'sys' is not defined, exit code: 1

In [112]:
print(tuner.latest_tuning_job.job_name)
boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
) #tensorflow-training-220627-0556

tensorflow-training-220627-0556


{'HyperParameterTuningJobName': 'tensorflow-training-220627-0556',
 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:949263681218:hyper-parameter-tuning-job/tensorflow-training-220627-0556',
 'HyperParameterTuningJobConfig': {'Strategy': 'Bayesian',
  'HyperParameterTuningJobObjective': {'Type': 'Minimize',
   'MetricName': 'loss'},
  'ResourceLimits': {'MaxNumberOfTrainingJobs': 9,
   'MaxParallelTrainingJobs': 3},
  'ParameterRanges': {'IntegerParameterRanges': [],
   'ContinuousParameterRanges': [{'Name': 'learning_rate',
     'MinValue': '0.0001',
     'MaxValue': '1',
     'ScalingType': 'Logarithmic'}],
   'CategoricalParameterRanges': []},
  'TrainingJobEarlyStoppingType': 'Off'},
 'TrainingJobDefinition': {'StaticHyperParameters': {'_tuning_objective_metric': 'loss',
   'model_dir': '"/opt/ml/model"',
   'sagemaker_container_log_level': '20',
   'sagemaker_estimator_class_name': '"TensorFlow"',
   'sagemaker_estimator_module': '"sagemaker.tensorflow.estimator"',
   's

# Contents of the recorded experiment

In [6]:
trial_component_analytics = ExperimentAnalytics(
    experiment_name='cifar-10-dataset-experiment',
    sort_by="metrics.test:accuracy.max",
    sort_order="Descending",
    metric_names=['test:accuracy'],
   # parameter_names=['hidden_channels', 'epochs', 'dropout', 'optimizer']
)
analytic_table = trial_component_analytics.dataframe()
analytic_table.head()

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,epochs,model_dir,sagemaker_container_log_level,...,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments,normalization_mean,normalization_std,cifar-10-dataset-log - MediaType,cifar-10-dataset-log - Value
0,cnn-training-job-1655879605-aws-training-job,Training,arn:aws:sagemaker:us-east-1:949263681218:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/t...,1.0,ml.m5.4xlarge,30.0,3.0,"""/opt/ml/model""",20.0,...,NaN,s3://sagemaker-us-east-1-949263681218/cv-model...,NaN,s3://sagemaker-us-east-1-949263681218/cv-model...,[cnn-training-job-32-hidden-channels-1655871250],[cifar-10-dataset-experiment],NaN,NaN,NaN,NaN
1,TrialComponent-2022-06-22-041210-guap,Preprocessing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,[cnn-training-job-10-hidden-channels-165587124...,"[cifar-10-dataset-experiment, cifar-10-dataset...",0.1307,0.3081,s3/uri,random
2,cnn-training-job-1655871262-aws-training-job,Training,arn:aws:sagemaker:us-east-1:949263681218:train...,763104351884.dkr.ecr.us-east-1.amazonaws.com/t...,1.0,ml.m5.2xlarge,30.0,1.0,"""/opt/ml/model""",20.0,...,NaN,s3://sagemaker-us-east-1-949263681218/cv-model...,NaN,s3://sagemaker-us-east-1-949263681218/cv-model...,[cnn-training-job-32-hidden-channels-1655871250],[cifar-10-dataset-experiment],NaN,NaN,NaN,NaN


In [4]:
from sagemaker.analytics import TrainingJobAnalytics
from sagemaker.analytics import ExperimentAnalytics

In [60]:
analytics = TrainingJobAnalytics(training_job_name = 'cnn-training-job-1655871262', metric_names=['test:accuracy'])

In [75]:
analytics.__dict__ #['_cloudwatch'].list_metrics()

{'_sage_client': <botocore.client.SageMaker at 0x7f6cb0c3f8d0>,
 '_cloudwatch': <botocore.client.CloudWatch at 0x7f6cb13a02e8>,
 '_training_job_name': 'cnn-training-job-1655871262',
 '_start_time': None,
 '_end_time': None,
 '_period': 60,
 '_metric_names': ['test:accuracy'],
 '_dataframe': None,
 '_data': defaultdict(list, {}),
 '_time_interval': {'start_time': datetime.datetime(2022, 6, 22, 4, 15, 56, 247000, tzinfo=tzlocal()),
  'end_time': datetime.datetime(2022, 6, 22, 4, 20, 6, 584000, tzinfo=tzlocal())}}

# Experiment Cleanup

In [94]:
# def cleanup_sme_sdk(experiment):
#     for trial_summary in experiment.list_trials():
#         trial = Trial.load(trial_name=trial_summary.trial_name)
#         for trial_component_summary in trial.list_trial_components():
#             tc = TrialComponent.load(
#                 trial_component_name=trial_component_summary.trial_component_name)
#             trial.remove_trial_component(tc)
#             try:
#                 # comment out to keep trial components
#                 tc.delete()
#             except:
#                 # tc is associated with another trial
#                 continue
#             # to prevent throttling
#             time.sleep(.5)
#         trial.delete()
#         experiment_name = experiment.experiment_name
#     experiment.delete()
#     print(f"\nExperiment {experiment_name} deleted")

In [ ]:
# cleanup_sme_sdk('cifar-10-dataset-experiment')

# LOAD TRAINED MODEL

In [8]:
model_path = f's3://{BUCKET}/cv-models/cifar_10/out/cnn-training-job-1655978931/output/model.tar.gz'
# model_path = f's3://{BUCKET}/cifar_10/out/tensorflow-training-220627-0556-001-17c5ce0a/output/model.tar.gz'
from sagemaker.tensorflow.model import TensorFlowModel

model = TensorFlowModel(model_data=model_path, role=role, framework_version="2.3.0")

# Deploy

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig

data_capture_prefix = "{}/monitoring/datacapture/".format(MONITORING_FOLDER)
s3_capture_upload_path = "s3://{}/{}".format(BUCKET, data_capture_prefix)

data_capture_configuration=DataCaptureConfig(
        enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
    )

In [52]:
endpoint_name=f'tensorflow-cv-{int(time.time())}'
predictor = tuner.deploy(initial_instance_count=1,
            #model.deploy(initial_instance_count=1,
                       instance_type='ml.m5.xlarge',
                       endpoint_name=endpoint_name,
 data_capture_config = data_capture_configuration)
print(f"\nSuccessfully deployed at {endpoint_name}...")


2022-06-27 06:00:30 Starting - Preparing the instances for training
2022-06-27 06:00:30 Downloading - Downloading input data
2022-06-27 06:00:30 Training - Training image download completed. Training in progress.
2022-06-27 06:00:30 Uploading - Uploading generated training model
2022-06-27 06:00:30 Completed - Training job completed

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.



-----------!
Successfully deployed at tensorflow-cv-1656312038...


In [101]:
import boto3
sm.describe_endpoint(EndpointName = endpoint_name)

{'EndpointName': 'tensorflow-cv-1656312038',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:949263681218:endpoint/tensorflow-cv-1656312038',
 'EndpointConfigName': 'tensorflow-cv-1656312038',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.0-cpu',
     'ResolvedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference@sha256:91ebb7428846c5f7b515d5d9b8389a14c73d0c5d02657f4a6413592124333278',
     'ResolutionTime': datetime.datetime(2022, 6, 27, 6, 40, 40, 775000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'DataCaptureConfig': {'EnableCapture': True,
  'CaptureStatus': 'Started',
  'CurrentSamplingPercentage': 100,
  'DestinationS3Uri': 's3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/datacapture/'},
 'EndpointStatus': 'InService',
 'CreationTim

# Invoking the Deployed Model

In [53]:
%matplotlib inline
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing import image
from IPython.display import Image
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import numpy as np
CIFAR10_LABELS = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [54]:
from sagemaker.tensorflow.model import TensorFlowPredictor
predictor = TensorFlowPredictor(endpoint_name = endpoint_name)

In [55]:
img = load_img('./data/cifar_10/raw_images/jeep.png', target_size=(32, 32))
data = img_to_array(img)
data = data.astype('float32')
data = data / 255.0
data = data.reshape(1, 32, 32, 3)


In [56]:
print(data.shape)

(1, 32, 32, 3)


In [57]:
payload = {'instances': data}

In [58]:
resp = predictor.predict(payload)
predicted_label = CIFAR10_LABELS[np.argmax(resp['predictions'])]
print(f'Predicted Label: [{predicted_label}]')

Predicted Label: [automobile]


In [27]:
!ls /opt/ml

metadata


# Sending Test Traffic

In [111]:
%%time

import time
print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

flat_list = []
for i in range(2):
    data = np.array([X_test[i]])
    payload = {'instances': data}
    resp = predictor.predict(payload)
    predicted_label = CIFAR10_LABELS[np.argmax(resp['predictions'])]
    flat_list.append(predicted_label)
    time.sleep(0.5)

print("Done!")
print("predictions: \t{}".format(np.array(flat_list)))

Sending test traffic to the endpoint tensorflow-cv-1656312038. 
Please wait...
Done!
predictions: 	['frog' 'automobile']
CPU times: user 21.7 ms, sys: 0 ns, total: 21.7 ms
Wall time: 1.08 s


In [38]:
s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=BUCKET, Prefix=data_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))
print(BUCKET)

Found Capture Files:
DEMO-tf2-ModelMonitor/monitoring/datacapture/
sagemaker-us-east-1-949263681218


# Prepare baseline Dataset

In [22]:
validate_dataset = "validation_with_predictions.csv"

In [ ]:
# X_train, y_train
# ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
data = np.array([X_test[i]])
    payload = {'instances': data}
    resp = predictor.predict(payload)
    predicted_label = CIFAR10_LABELS[np.argmax(resp['predictions'])]

In [27]:
i = 0
with open(f"{validate_dataset}", "w") as baseline_file:
    baseline_file.write("probability,prediction,label\n")  # our header
    for i in range(1000):
        data = np.array([X_train[i]])
        payload = {'instances': data}
        resp = predictor.predict(payload)
        probability = max(resp['predictions'][0])
        prediction = np.argmax(resp['predictions'])
        label = y_train[i][0]
        
        baseline_file.write(f"{probability},{prediction},{label}\n")
print("Done!")

Done!


In [28]:
!aws s3 cp ./validation_with_predictions.csv s3://{BUCKET}/DEMO-tf2-ModelMonitor/monitoring/baseline/

upload: ./validation_with_predictions.csv to s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/baseline/validation_with_predictions.csv


# MODEL MONITORING

In [102]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

In [103]:
my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

In [105]:
# GENERATE constraints and statistics
baseline_data_uri = f's3://{BUCKET}/{MONITORING_FOLDER}/monitoring/baseline/'
baseline_data_output_uri = f's3://{BUCKET}/{MONITORING_FOLDER}/monitoring/baseline/output'
my_default_monitor.suggest_baseline(
    job_name = 'test-baseline-job-newest',
    baseline_dataset=baseline_data_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_data_output_uri,
    wait=True
)


Job Name:  test-baseline-job-newest
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/baseline/', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/baseline/output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
............................2022-06-27 07:31:16,452 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2022-06-27 07:31:16.973283: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: 

In [31]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,probability,Fractional,1000,0,0.403413,403.412881,0.178749,0.116331,0.974984,"[{'lower_bound': 0.116330899, 'upper_bound': 0...",0.64,2048.0,"[[0.418848723, 0.293665916, 0.476312459, 0.483..."
1,prediction,Integral,1000,0,5.209000,5209.000000,2.790577,0.000000,9.000000,"[{'lower_bound': 0.0, 'upper_bound': 0.9, 'cou...",0.64,2048.0,"[[7.0, 1.0, 5.0, 8.0, 0.0, 9.0, 1.0, 1.0, 9.0,..."
2,label,Integral,1000,0,4.583000,4583.000000,2.901226,0.000000,9.000000,"[{'lower_bound': 0.0, 'upper_bound': 0.9, 'cou...",0.64,2048.0,"[[5.0, 8.0, 2.0, 8.0, 0.0, 4.0, 1.0, 8.0, 9.0,..."


In [32]:
constraints_df = pd.io.json.json_normalize(
    baseline_job.suggested_constraints().body_dict["features"]
)
constraints_df.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,inferred_type,completeness,num_constraints.is_non_negative
0,probability,Fractional,1.0,True
1,prediction,Integral,1.0,True
2,label,Integral,1.0,True


In [38]:
%%writefile preprocessing.py

import json
def preprocess_handler(inference_record):
    input_data = json.loads(inference_record.endpoint_input.data)
    input_data = {f"feature{str(i).zfill(10)}": val for i, val in enumerate(input_data)}

    output_data = json.loads(inference_record.endpoint_output.data)["predictions"][0][0]
    output_data = {"prediction0": output_data}

    return {**input_data}

Writing preprocessing.py


In [33]:
preprocessor_s3_dest_path = f"s3://{BUCKET}/{MONITORING_FOLDER}/monitoring/preprocessor"
preprocessor_s3_dest = sagemaker.s3.S3Uploader.upload("preprocessing.py", preprocessor_s3_dest_path)
print(preprocessor_s3_dest)

s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/preprocessor/preprocessing.py


In [106]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

s3_report_path = 's3://{BUCKET}/{MONITORING_FOLDER}/monitoring/preprocessor/processed_output'
mon_schedule_name = "DEMO-tf2-model-monitor-schedule-" + strftime("%Y%m%d-%H%M%S", gmtime())
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    record_preprocessor_script=preprocessor_s3_dest,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## Exploring Generated constraints

In [ ]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

In [ ]:
constraints_df = pd.io.json.json_normalize(
    baseline_job.suggested_constraints().body_dict["features"]
)
constraints_df.head(10)

# Delete Endpoint

In [84]:
import boto3
sm = boto3.client('sagemaker')

In [86]:
endpoint_name

'tensorflow-cv-1656312038'

In [85]:
sm.list_endpoints()

{'Endpoints': [{'EndpointName': 'tensorflow-cv-1656312038',
   'EndpointArn': 'arn:aws:sagemaker:us-east-1:949263681218:endpoint/tensorflow-cv-1656312038',
   'CreationTime': datetime.datetime(2022, 6, 27, 6, 40, 38, 793000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2022, 6, 27, 6, 45, 51, 665000, tzinfo=tzlocal()),
   'EndpointStatus': 'InService'},
  {'EndpointName': 'tensorflow-cv-1656302831',
   'EndpointArn': 'arn:aws:sagemaker:us-east-1:949263681218:endpoint/tensorflow-cv-1656302831',
   'CreationTime': datetime.datetime(2022, 6, 27, 4, 7, 12, 465000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2022, 6, 27, 4, 13, 30, 653000, tzinfo=tzlocal()),
   'EndpointStatus': 'InService'}],
 'ResponseMetadata': {'RequestId': '3b6b05b3-a849-49dd-83bf-2691fa56e898',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3b6b05b3-a849-49dd-83bf-2691fa56e898',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '479',
   'date': 'Mon,

In [87]:
endpoint_name='tensorflow-cv-1656302831'
sm.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '24a2674f-bfc2-49ac-9dec-99c2b915321c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '24a2674f-bfc2-49ac-9dec-99c2b915321c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 27 Jun 2022 07:03:55 GMT'},
  'RetryAttempts': 0}}

In [88]:
endpoint_name = 'tensorflow-cv-1656312038'

# TEST CODES

In [112]:
predictor

In [113]:
data_capture_config

In [116]:
data_capture_config.destination_s3_uri

's3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/datacapture'

In [113]:
sm.list_monitoring_schedules()

{'MonitoringScheduleSummaries': [{'MonitoringScheduleName': 'DEMO-tf2-model-monitor-schedule-20220627-073726',
   'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:949263681218:monitoring-schedule/demo-tf2-model-monitor-schedule-20220627-073726',
   'CreationTime': datetime.datetime(2022, 6, 27, 7, 37, 27, 456000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2022, 6, 27, 9, 6, 12, 538000, tzinfo=tzlocal()),
   'MonitoringScheduleStatus': 'Scheduled',
   'EndpointName': 'tensorflow-cv-1656312038',
   'MonitoringJobDefinitionName': 'data-quality-job-definition-2022-06-27-07-37-27-181',
   'MonitoringType': 'DataQuality'}],
 'ResponseMetadata': {'RequestId': 'd079186d-8972-482d-81bd-947dae6efc23',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd079186d-8972-482d-81bd-947dae6efc23',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '509',
   'date': 'Mon, 27 Jun 2022 09:10:56 GMT'},
  'RetryAttempts': 0}}

In [114]:
sm.describe_monitoring_schedule(MonitoringScheduleName = 'DEMO-tf2-model-monitor-schedule-20220627-073726')

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:949263681218:monitoring-schedule/demo-tf2-model-monitor-schedule-20220627-073726',
 'MonitoringScheduleName': 'DEMO-tf2-model-monitor-schedule-20220627-073726',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'DataQuality',
 'CreationTime': datetime.datetime(2022, 6, 27, 7, 37, 27, 456000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 6, 27, 9, 6, 12, 538000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'data-quality-job-definition-2022-06-27-07-37-27-181',
  'MonitoringType': 'DataQuality'},
 'EndpointName': 'tensorflow-cv-1656312038',
 'LastMonitoringExecutionSummary': {'MonitoringScheduleName': 'DEMO-tf2-model-monitor-schedule-20220627-073726',
  'ScheduledTime': datetime.datetime(2022, 6, 27, 9, 0, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2022, 6, 27, 9, 5, 50, 205000, tzinfo=tzlocal()

In [83]:
sm.delete_monitoring_schedule(MonitoringScheduleName = 'DEMO-tf2-model-monitor-schedule-2022-06-27-04-43-15')

{'ResponseMetadata': {'RequestId': '74d73803-5125-43ad-a548-729023f93df1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '74d73803-5125-43ad-a548-729023f93df1',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 27 Jun 2022 07:03:05 GMT'},
  'RetryAttempts': 0}}

In [ ]:
https://stackoverflow.com/questions/69179914/how-to-fix-sagemaker-data-quality-monitoring-schedule-job-that-fails-with-failu

In [16]:
https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_model_monitor/fairness_and_explainability/SageMaker-Model-Monitor-Fairness-and-Explainability.html

In [66]:
!cp /opt/ml/input_data

In [67]:
!aws s3 cp ./{DATASET_PATH}/X_train.npy s3://{BUCKET}/{PREFIX}/cifar_10/train/

fatal error: An error occurred (404) when calling the HeadObject operation: Key "DEMO-tf2-ModelMonitor/monitoring/ground_truth/" does not exist


In [77]:
!aws s3 cp s3://sagemaker-us-east-1-949263681218/DEMO-tf2-ModelMonitor/monitoring/ground_truth/2022/ /opt/ml/
    

fatal error: An error occurred (404) when calling the HeadObject operation: Key "DEMO-tf2-ModelMonitor/monitoring/ground_truth" does not exist


In [79]:
!ls /opt/ml/